In [3]:
%load_ext autoreload
%autoreload 2

import os, re, json, ast
from llm_api_calls import send_message_to_gemini_async
from km_utils import get_ToCs, split_dict_by_size, get_content_from_articles_response, split_list_by_size
from coap_prompts import *
from parallel_descriptions import RateLimiter
from pprint import pprint

# rate_limiter = RateLimiter(9, 10)
rate_limiter = RateLimiter(13, 14)
# rate_limiter = RateLimiter(1, 10)

# Pipeline

In [4]:
ToCs = get_ToCs(base_path = 'coap_map')

doc_with_childs = ToCs['doc_with_childs']
ToC_sections = ToCs['ToC_sections']
ToC_chapters = ToCs['ToC_chapters']
ToC_chapters_with_article_short_descriptions = ToCs['ToC_chapters_with_article_short_descriptions']
ToC_articles = ToCs['ToC_articles']
ToC_articles_short_descriptions = ToCs['ToC_articles_short_descriptions']
ToC_articles_descriptions = ToCs['ToC_articles_descriptions']
ToC_sections_articles_shortDescription = ToCs['ToC_sections_articles_shortDescription']

In [6]:
user_query_ori = "Что будет за жестокое обращение с животным"

user_query = await get_perefrased_query(user_query_ori)
user_query = user_query['text_response']
print(f'Изначальный вопрос: {user_query_ori}')
print(f'Уточняющие вопросы: {user_query}')

get_section_response = await get_section_nagigation(user_query_ori, user_query, ToC_sections)
choosed_sections = parse_section_response(get_section_response, ToC_articles)

total_articles = []
for section in choosed_sections:
    print('section:', section)
    ToC_article_chunks = split_list_by_size(list(ToC_sections_articles_shortDescription[section].items()), size_limit=26000)
    choosed_articles_responses = await get_chunked_ARTICLE_navigation(user_query_ori, user_query, ToC_article_chunks, rate_limiter)
    articles = get_content_from_articles_response(ToC_articles, choosed_articles_responses)
    print(f'Найдено {len(articles)} статей')
    total_articles.extend(articles)


print(f'Найдено всего {len(total_articles)} статей')
print('***')
pprint([a['article_title'] for a in total_articles], width=300)
print('***')

context = [f"{x['article_title']} : {x['content']}" for x in total_articles]
if len(str(context)) > 25000:
    context = choose_articles_second_round_and_limit_context(context, user_query_ori, user_query, ToC_articles, size_limit=25000, rate_limiter=rate_limiter)

get_article_response, prompt = await get_final_response(user_query_ori, user_query, 
                                                        context,
                                                        rate_limiter=rate_limiter)
print('\n\n ФИНАЛЬНЫЙ ОТВЕТ: ***')
print(get_article_response.get('text_response'))


Изначальный вопрос: Что будет за жестокое обращение с животным
Уточняющие вопросы: ["Какую ответственность влечет за собой совершение административного правонарушения, выраженного в жестоком обращении с животным?", "Каков размер штрафа за жестокое обращение с животным в соответствии с КоАП РФ?", "Какое наказание предусмотрено за причинение боли животному в рамках административного правонарушения?"]
section: Раздел II. Особенная часть
chapter_article_map: 
Статья 10.5.1.
ничего не найдено
ничего не найдено
ничего не найдено
ничего не найдено
ничего не найдено
ничего не найдено
Статья 6.16.1.
Статья 6.8.
ничего не найдено
Статья 6.16.1.
ничего не найдено
Статья 245.
Статья 245.1.
ничего не найдено
Статья 10.5.1.
Статья 6.16.1.
Статья 6.8.
Статья 6.16.1.
Статья 245.
Статья 245.1.
Найдено 3 статей
Найдено всего 3 статей
***
['Статья 10.5.1. Незаконное культивирование растений, содержащих наркотические средства или психотропные вещества либо их прекурсоры',
 'Статья 6.16.1. Незаконные приоб

In [6]:
get_article_response, prompt = await get_final_response(user_query_ori, user_query, 
                                                        context,
                                                        rate_limiter=rate_limiter)
print('\n\n ФИНАЛЬНЫЙ ОТВЕТ: ***')
print(get_article_response.get('text_response'))




 ФИНАЛЬНЫЙ ОТВЕТ: ***
**Ответ на основной вопрос:**

Штраф за осуществление предпринимательской деятельности без государственной регистрации в качестве индивидуального предпринимателя составляет от 500 до 2000 рублей.

**Примечание:**

* В статье 14.1 КоАП РФ указано, что осуществление предпринимательской деятельности без государственной регистрации в качестве индивидуального предпринимателя влечет наложение административного штрафа в размере от 500 до 2000 рублей.
* Статья 14.1.2 КоАП РФ предусматривает штраф в размере 50 000 рублей за осуществление предпринимательской деятельности в области транспорта без лицензии.
* Статья 19.20 КоАП РФ устанавливает штраф от 500 до 1000 рублей за осуществление деятельности, не связанной с извлечением прибыли, без специального разрешения (лицензии).

**Использованные статьи:**

* Статья 14.1 КоАП РФ
* Статья 14.1.2 КоАП РФ
* Статья 19.20 КоАП РФ


In [ ]:
# какой штраф за предпринимательскую деятельность без ИП?

In [ ]:

test_cases = {
    'user_query_ori': "что будет за дискредитацию армии рф?", 'needed_articles': ["Статья 20.3.3."],
    'user_query_ori': "штраф за пересечение стоп-линии красный сигнал светофора", 'needed_articles': ["Статья 12.12."],
    'user_query_ori': "какое наказание за марихуану?", 'needed_articles': ["Статья 6.8."],
    'user_query_ori': "какое наказание за драку?", 'needed_articles': ["Статья 20.1.", "Статья 6.1.1."],
    'user_query_ori': "что будет если скрыться с места дтп?", 'needed_articles': ["Статья 12.27."],
    'user_query_ori': "какой штраф за просроченный паспорт?", 'needed_articles': ["Статья 19.15."],
    'user_query_ori': "что будет за дачу взятки?", 'needed_articles': ["Статья 19.28."],
    'user_query_ori': "что будет за вырубку леса?", 'needed_articles': ["Статья 8.25."],
    'user_query_ori': "какой штраф за порчу чужого имущества?", 'needed_articles': ["Статья 7.17."],
    'user_query_ori': "какой штраф за вождение без прав?", 'needed_articles': ["Статья 12.7."],
    'user_query_ori': "что будет за неявку по повестке?", 'needed_articles': ["Статья 27.15.", "Статья 21.5."],
    'user_query_ori': "что будет за тонировку?", 'needed_articles': ["Статья 12.5."],
    'user_query_ori': "Что будет за дискриминацию по полу, цвету кожи, расе?", 'needed_articles': ["Статья 5.62."],
    'user_query_ori': "Что будет за сокрытие ВИЧ?", 'needed_articles': ["Статья 6.1."],
    'user_query_ori': "Что будет за потребление наркотических веществ", 'needed_articles': ["Статья 20.20.", "Статья 6.9."],
    'user_query_ori': "Что будет за повреждение памятников культурного наследия", 'needed_articles': ["Статья 7.13."],
    'user_query_ori': "какой штраф за предпринимательскую деятельность без ИП?", 'needed_articles': ["Статья 14.1."],
    'user_query_ori': "Что будет, если не уступить дорогу пешеходу?", 'needed_articles': ["Статья 12.18."],



    
              }